Mat imread(const string &filename, int flags=1)
flags:
opencv 2.x
CV_LOAD_IMAGE_COLOR
CV_LOAD_IMAGE_GRAYSCALE
CV_LOAD_IMAGE_ANYCOLOR
opencv 3.x
IMREAD_COLOR
IMREAD_GRAYSCALE
IMREAD_ANYCOLOR

图像的几何变换

完成一张图像的几何变换需要两个算法：1）实现空间坐标变换，描述每个像素如何从初始位置移动到终止位置 2）插值算法，完成输出图像每个像素点的灰度值。主要有 仿射变换、投影变换、极坐标变换。

仿射变换由以下公式描述：
$$\begin{bmatrix} x^{*}\\ y^{*}\end{bmatrix}
=\begin{bmatrix}a_{11}& a_{12} \\
a_{21}& a_{22} \end{bmatrix}
\begin{bmatrix} x\\ y\end{bmatrix}
+ \begin{bmatrix}a_{13}\\ a_{23}\end{bmatrix}
$$<br>
$$
x^* = a_{11}\cdot x + a_{12}\cdot y + a_{13}, y^* = a_{21}\cdot x + a_{22}\cdot y + a_{23}
$$

用一个矩阵乘法来表示：
$$\begin{bmatrix} x^{*}\\ y^{*}\\1\end{bmatrix}
=\begin{bmatrix}a_{11}& a_{12}& a_{13} \\
a_{21}& a_{22}& a_{23} \\
0& 0& 1\end{bmatrix}
\begin{bmatrix} x\\ y\\ 1\end{bmatrix}
$$
$A=\begin{bmatrix}a_{11}& a_{12}& a_{13} \\
a_{21}& a_{22}& a_{23} \\
0& 0& 1\end{bmatrix}$
称为仿射变换矩阵。在讨论中经常忽略第三行。

平移：
$$\begin{bmatrix} x^{*}\\ y^{*}\\1\end{bmatrix}
=\begin{bmatrix}1& 0& t_x \\
0& 1& t_y \\
0& 0& 1\end{bmatrix}
\begin{bmatrix} x\\ y\\ 1\end{bmatrix}
$$
表示沿$x$方向平移$t_x$个像素，沿$y$方向平移$t_y$个像素。

缩放：
$$\begin{bmatrix} x^{*}\\ y^{*}\\1\end{bmatrix}
=\begin{bmatrix}s_{x}& 0& 0 \\
0& s_{y}& 0 \\
0& 0& 1\end{bmatrix}
\begin{bmatrix} x\\ y\\ 1\end{bmatrix}
$$
上式为以原点为中心的缩放， 若要以任何点$(x_0, y_0)$为中心缩放，则以下式表示：
$$\begin{bmatrix} x^{*}\\ y^{*}\\1\end{bmatrix}
=\begin{bmatrix}1& 0& x_0 \\
0& 1& y_0 \\
0& 0& 1\end{bmatrix}
\begin{bmatrix}s_{x}& 0& 0 \\
0& s_{y}& 0 \\
0& 0& 1\end{bmatrix}
\begin{bmatrix}1& 0& -x_0 \\
0& 1& -y_0 \\
0& 0& 1\end{bmatrix}
\begin{bmatrix} x\\ y\\ 1\end{bmatrix}
$$

旋转：
$$\begin{bmatrix} x^{*}\\ y^{*}\\1\end{bmatrix}
=\begin{bmatrix}cos{\alpha}& sin{\alpha}& 0 \\
-sin{\alpha}& cos{\alpha}& 0 \\
0& 0& 1\end{bmatrix}
\begin{bmatrix} x\\ y\\ 1\end{bmatrix}
$$
上式为以原点为中心的旋转， 若要以任何点$(x_0, y_0)$为中心旋转，则以下式表示：
$$\begin{bmatrix} x^{*}\\ y^{*}\\1\end{bmatrix}
=\begin{bmatrix}1& 0& x_0 \\
0& 1& y_0 \\
0& 0& 1\end{bmatrix}
\begin{bmatrix}cos{\alpha}& sin{\alpha}& 0 \\
-sin{\alpha}& cos{\alpha}& 0 \\
0& 0& 1\end{bmatrix}
\begin{bmatrix}1& 0& -x_0 \\
0& 1& -y_0 \\
0& 0& 1\end{bmatrix}
\begin{bmatrix} x\\ y\\ 1\end{bmatrix}
$$

如果已知变换前后点的坐标，只需要有3对，则可以通过方程求出仿射变换矩阵。opencv中提供了$getAffineTransform(src, dst)$函数来计算仿射矩阵。

当仿射矩阵由多个矩阵组合而成时，可以采用numpy的$np.dot(t, s)$来计算，也可以通过opencv的“*”运算符或者$gemm()$函数来实现。

前面描述了坐标变换的仿射矩阵。在实际操作时，往往需要已知目标坐标的情况下，寻找源坐标。
$$\begin{bmatrix} x\\ y\\1\end{bmatrix}
=A^{-1}
\begin{bmatrix} x^{*}\\ y^{*}\\1\end{bmatrix}
$$

对每一个新坐标的函数值，均需要从源坐标处得到。因为源坐标均为整型，而计算的出来的坐标值可能为浮点型，或者落在原有坐标范围之外，则需要定义插值算法。

一般坐标值在范围之外使用固定值填充。常用的插值算法有：最近邻插值、双线性插值、三次样条插值。最近邻插值的结果容易出现锯齿状，双线性插值比较常用。
opencv提供$warpAffine()$函数，输入仿射变换矩阵，来计算仿射变换的结果。

为了使用方便，省去创建仿射矩阵的步骤，opencv提供$resize()， rotate()$两个函数，实现图像的缩放和旋转。

卷积：

一维离散卷积：

  简单的说，就是反折对位相加再移位。$s$有m个元素，$t$有n个元素，$s*n$的结果具有m+n-1个元素。

二维离散卷积：

  二维与一维类似。二维卷积表示为两个矩阵的运算。较小的矩阵我们称为卷积核。首先把卷积核逆时针翻转180度。
然后按照先行后列的顺序，每移动到一个位置就对应相乘再相加。这样会生成一个新的矩阵。矩阵大小$H_1 + H_2 - 1 * W_1 + W_2 - 1$
这样计算的卷积也称为full卷积。

valid卷积：

  从full卷积的计算过程可以看出，卷积核在边界时，会超出图像边界，访问未定义的区域。如果不考虑这种情况，只考虑能完全覆盖的情况，则称为valid卷积。
  很明显，valid卷积是full卷积的一部分。大小为$H_1 - H_2 + 1 * W_1 - W_2 +1$
  
same卷积：

很明显full卷积核valid卷积的结果与原始图像的大小均不相同。为了使卷积结果和原始图像大小相等，在卷积核中定义一个锚点，当锚点遍历图像的每一个点所得到的结果成为same卷积。

锚点可以是卷积核中的任何点，如果卷积核的长宽均为奇数，则一般选择中心点。

边界填充方法：

1. 填充常数值，一般为0。BORDER_CONSTANT
2. 复制边界处的值。BORDER_REPLICATE
3. 平铺。 BORDER_WRAP
4. 以边界为中心，反射。BORDER_REFLECT, BORDER_REFLECT_101

$opencv提供copyMakeBorder()$函数来填充边界。

$scipy提供convolve2d()$函数来计算卷积。如果需要计算任意锚点的卷积，则需要先计算full卷积，然后再从结果中提取。

$opencv中没有提供类似的函数，但可以使用flip()和filter2D()结合来完成$。$flip()$完成卷积核的翻转。
